# Annotating Training Data With MTurk

## Pre-requisites
If you haven't already, you'll need to setup MTurk and AWS accounts that are linked together to use MTurk with Python. The MTurk account will be used to post tasks to the MTurk crowd and the AWS accounts will be used to connect to MTurk via API and provide access to any additional AWS resources that are needed to execute your task.

1. If you don't have an AWS account already, visit https://aws.amazon.com and create an account you can use for your project.
2. If you don't have an MTurk Requester account already, visit https://requester.mturk.com and create a new account.

After you've setup your accounts, you will need to link them together. When logged into both the root of your AWS account and your MTurk account, visit https://requester.mturk.com/developer to link them together.

From your AWS console create a new AWS IAM User or select an existing one you plan to use. Add the AmazonMechanicalTurkFullAccess policy to your user. Then select the Security Credentials tab and create a new Access Key, copy the Access Key and Secret Access Key for future use.

If you haven't installed the awscli yet, install it with pip (pip install awscli) and configure a profile using the access key and secret key above (aws configure --profile mturk). 

To post tasks to MTurk for Workers to complete you will first need to add funds to your account that will be used to reward Workers. Visit https://requester.mturk.com/account to get started with as little as $1.00.

We also recommend installing xmltodict as shown below.

In [ ]:
!pip install boto3

In [ ]:
!pip install xmltodict

## Overview
Amazon Mechanical Turk allows you to post tasks for Workers to complete at https://worker.mturk.com. To post a task to
MTurk you create an HTML form that includes the information you want them to provide. In this example we'll be asking Workers to rate the sentiment of Tweets on a scale of 1 (negative) to 10 (positive).

MTurk has a Sandbox environment that can be used for testing. Workers won't work see your tasks in the Sandbox but you can log in to do them yourself to test the task interface at https://workersandbox.mturk.com. It's recommended you test first in the Sandbox to make sure your task returns the data you need before moving to the Production environment. There is no cost to use the Sandbox environment.

In [1]:
import boto3
import xmltodict
import json
import os
from datetime import datetime
import random
import pandas as pd 
import csv

In [2]:
create_hits_in_production = True
environments = {
        "production": {
            "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
            "preview": "https://www.mturk.com/mturk/preview"
        },
        "sandbox": {
            "endpoint": "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
            "preview": "https://workersandbox.mturk.com/mturk/preview"
        },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]

session = boto3.Session(profile_name='mturk')

client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

In [56]:
# This will return your current MTurk balance if you are connected to Production.
# If you are connected to the Sandbox it will return $10,000.
print(client.get_account_balance()['AvailableBalance'])

354.45


## Define your task
For this project we are going to get the sentiment of a set of tweets that we plan to train a model to evaluate. We will create an MTurk Human Intelligence Task (HIT) for each tweet.

## Handle the combination of pics

In [41]:
survey_groups = pd.read_csv('survey_groups.csv') 
imagePath = "https://my-image-repo-520.s3.amazonaws.com/uploads"
# Preview the first 5 lines of the loaded data 
t = survey_groups[10:25]
for index, row in t.iterrows():
    for i, col in enumerate(survey_groups.columns): 
        imgName  = row[col]
        print(index, i,row[col],"{}/{}".format(imagePath,imgName.strip().replace(" ","+")))

10 0 Alternet Page - Adidas.png https://my-image-repo-520.s3.amazonaws.com/uploads/Alternet+Page+-+Adidas.png
10 1 Alternet Page - Polar Bear.png https://my-image-repo-520.s3.amazonaws.com/uploads/Alternet+Page+-+Polar+Bear.png
10 2 Book - Liberal Quiz.png https://my-image-repo-520.s3.amazonaws.com/uploads/Book+-+Liberal+Quiz.png
10 3 Book - Fasion.png https://my-image-repo-520.s3.amazonaws.com/uploads/Book+-+Fasion.png
10 4 Breitbart Page - Fasion.png https://my-image-repo-520.s3.amazonaws.com/uploads/Breitbart+Page+-+Fasion.png
10 5 Breitbart Page - Wildlife.png https://my-image-repo-520.s3.amazonaws.com/uploads/Breitbart+Page+-+Wildlife.png
10 6 Daily Kos Page - bp.png https://my-image-repo-520.s3.amazonaws.com/uploads/Daily+Kos+Page+-+bp.png
10 7 Daily Kos Page - Polar Bear.png https://my-image-repo-520.s3.amazonaws.com/uploads/Daily+Kos+Page+-+Polar+Bear.png
10 8 Food - Impeach.png https://my-image-repo-520.s3.amazonaws.com/uploads/Food+-+Impeach.png
10 9 Food - Fasion.png https:/

MTurk accepts an XML document containing the HTML that will be displayed to Workers. Workers will see these HTML for each item tweet that is submitted. To use the HTML for this example task, download it from [here](https://s3.amazonaws.com/mturk/samples/jupyter-examples/SentimentQuestion.html) and store it in the same directory as this notebook. Within the HTML is a variable ${content} that will be replaced with a different tweet when the HIT is created.

Here the HTML is loaded and inserted into the XML Document.

In [42]:
html_layout = open('./survey.html', 'r',encoding="utf-8").read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

In Mechanical Turk each task is representated by a Human Intelligence Task (HIT) which is an individual item you want annotated by one or more Workers and the interface that should be displayed. The definition below requests that five Workers review each item, that the HIT remain live on the worker.mturk.com website for no more than an hour, and that Workers provide a response for each item in less than ten minutes. Each response has a reward of \$0.05 so the total Worker reward for this task would be \$0.25 plus \$0.05 in MTurk fees. An appropriate title, description, keywords are also provided to let Workers know what is involved in this task.

In [43]:
TaskAttributes = {
    'MaxAssignments': 9,                 
    'LifetimeInSeconds': 60*60*24*7,           # How long the task will be available on the MTurk website (7 days)
    'AssignmentDurationInSeconds': 60*60*2, # How long Workers have to complete each item (2 Hours)
    'Reward': '0.24',                     # The reward you will offer Workers for each response
    'Title': 'Answer questions about ads',
    'Keywords': 'survey, ad, webpage, questionnaire',
    'Description': 'Rate the relevancy of an ad to a webpage from 1 to 5'
}


## Create the tasks
Here a HIT is created for each tweet so that it can be completed by Workers. Prior to creating the HIT, the tweet is inserted into the Question XML content. The HIT Id returned for each task is stored in a results array so that we can retrieve the results later.

In [44]:
survey_groups = pd.read_csv('survey_groups.csv') 
results = []
hit_type_id = ''
numberOFImage = 16
imagePath = "https://my-image-repo-520.s3.amazonaws.com/uploads"
slicedData = survey_groups[10:25]

for index, row in slicedData.iterrows():
    result = {}
    question = question_xml
    for i, col in enumerate(survey_groups.columns): 
        imgName  = row[col]
        to_split = imgName.replace('.png', '').replace('.PNG', '').replace('Page2', '').replace('Page', '').replace('Ad', '').replace('Book', 'NYT').replace('Food', 'NYT').replace('Fasion', 'Fashion').replace('bp', 'BP').replace('Impeach', 'Impeachment')

        pageName, adName = to_split.split(' - ')
        question = question.replace('${{url_{0}}}'.format(i+1), "{}/{}".format(imagePath, imgName.strip().replace(" ", "+")))
        question = question.replace('${{website_{0}}}'.format(i+1), pageName.strip())
        question = question.replace('${{ad_name_{0}}}'.format(i+1), adName.strip())
        result['image{}'.format(i+1)] = imgName
    response = client.create_hit(
        **TaskAttributes,
        Question = question
    )
    print(index+1)
    hit_type_id = response['HIT']['HITGroupId']
        
    result['id'] = index + 1
    result['hit_id'] = response['HIT']['HITId']
    results.append(result)

print("You can view the HITs here:")
print(mturk_environment['preview'] + "?groupId={}".format(hit_type_id))

if not os.path.exists("result/"):
    os.makedirs("result/")
    
now = datetime.now()

dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
with open('result/result-{}.json'.format(dt_string), 'w') as outfile:
    json.dump(results, outfile)

11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
You can view the HITs here:
https://www.mturk.com/mturk/preview?groupId=35QUA2ABZ77DYUFKKCTS92PV602EG6


## Block workers

In [46]:
if os.path.exists('workerIDs.json'):
    with open('workerIDs.json') as json_file:
        workerIDs = json.load(json_file)
    
    for wid in workerIDs:
        response = client.create_worker_block(
            WorkerId=wid,
            Reason='You already did this HIT.'
        )
    

## Delete worker block

In [ ]:
workerId = ""
response = client.create_worker_block(
    WorkerId=workerId,
    Reason='You are not block anymore.'
)

In [ ]:
results

## Get Results
Depending on the task, results will be available anywhere from a few minutes to a few hours. Here we retrieve the status of each HIT and the responses that have been provided by Workers.

In [8]:
def getAnsewer(answerDict):
    answer ={}
    for ans in answer_dict['QuestionFormAnswers']['Answer']:
        if ans['QuestionIdentifier'] == "age":
            answer["age"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "gender":
            answer["gender"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "race":
            answer["race"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "zipCode":
            answer["zipCode"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "Hispanic":
            answer["Hispanic"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "education":
            answer["education"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "occupation":
            answer["occupation"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "Political":
            answer["Political"] = ans["FreeText"]
                
        elif  ans['QuestionIdentifier'] == "d1.strong_disagree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 1
        elif  ans['QuestionIdentifier'] == "d1.disagree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 2
        elif  ans['QuestionIdentifier'] == "d1.Unsure" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 3
        elif  ans['QuestionIdentifier'] == "d1.agree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 4
        elif  ans['QuestionIdentifier'] == "d1.strong_agree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 5
            
        elif  ans['QuestionIdentifier'] == "q1.1" and ans["FreeText"] == "true":
            answer["q1"] = 1
        elif  ans['QuestionIdentifier'] == "q1.2" and ans["FreeText"] == "true":
            answer["q1"] = 2
        elif  ans['QuestionIdentifier'] == "q1.3" and ans["FreeText"] == "true":
            answer["q1"] = 3
        elif  ans['QuestionIdentifier'] == "q1.4" and ans["FreeText"] == "true":
            answer["q1"] = 4
        elif  ans['QuestionIdentifier'] == "q1.5" and ans["FreeText"] == "true":
            answer["q1"] = 5
            
        elif  ans['QuestionIdentifier'] == "q2.1" and ans["FreeText"] == "true":
            answer["q2"] = 1
        elif  ans['QuestionIdentifier'] == "q2.2" and ans["FreeText"] == "true":
            answer["q2"] = 2
        elif  ans['QuestionIdentifier'] == "q2.3" and ans["FreeText"] == "true":
            answer["q2"] = 3
        elif  ans['QuestionIdentifier'] == "q2.4" and ans["FreeText"] == "true":
            answer["q2"] = 4
        elif  ans['QuestionIdentifier'] == "q2.5" and ans["FreeText"] == "true":
            answer["q2"] = 5
            
        elif  ans['QuestionIdentifier'] == "q3.1" and ans["FreeText"] == "true":
            answer["q3"] = 1
        elif  ans['QuestionIdentifier'] == "q3.2" and ans["FreeText"] == "true":
            answer["q3"] = 2
        elif  ans['QuestionIdentifier'] == "q3.3" and ans["FreeText"] == "true":
            answer["q3"] = 3
        elif  ans['QuestionIdentifier'] == "q3.4" and ans["FreeText"] == "true":
            answer["q3"] = 4
        elif  ans['QuestionIdentifier'] == "q3.5" and ans["FreeText"] == "true":
            answer["q3"] = 5
            
        elif  ans['QuestionIdentifier'] == "q4.1" and ans["FreeText"] == "true":
            answer["q4"] = 1
        elif  ans['QuestionIdentifier'] == "q4.2" and ans["FreeText"] == "true":
            answer["q4"] = 2
        elif  ans['QuestionIdentifier'] == "q4.3" and ans["FreeText"] == "true":
            answer["q4"] = 3
        elif  ans['QuestionIdentifier'] == "q4.4" and ans["FreeText"] == "true":
            answer["q4"] = 4
        elif  ans['QuestionIdentifier'] == "q4.5" and ans["FreeText"] == "true":
            answer["q4"] = 5
            
        elif  ans['QuestionIdentifier'] == "q5.1" and ans["FreeText"] == "true":
            answer["q5"] = 1
        elif  ans['QuestionIdentifier'] == "q5.2" and ans["FreeText"] == "true":
            answer["q5"] = 2
        elif  ans['QuestionIdentifier'] == "q5.3" and ans["FreeText"] == "true":
            answer["q5"] = 3
        elif  ans['QuestionIdentifier'] == "q5.4" and ans["FreeText"] == "true":
            answer["q5"] = 4
        elif  ans['QuestionIdentifier'] == "q5.5" and ans["FreeText"] == "true":
            answer["q5"] = 5
            
        elif  ans['QuestionIdentifier'] == "q6.1" and ans["FreeText"] == "true":
            answer["q6"] = 1
        elif  ans['QuestionIdentifier'] == "q6.2" and ans["FreeText"] == "true":
            answer["q6"] = 2
        elif  ans['QuestionIdentifier'] == "q6.3" and ans["FreeText"] == "true":
            answer["q6"] = 3
        elif  ans['QuestionIdentifier'] == "q6.4" and ans["FreeText"] == "true":
            answer["q6"] = 4
        elif  ans['QuestionIdentifier'] == "q6.5" and ans["FreeText"] == "true":
            answer["q6"] = 5
            
        elif  ans['QuestionIdentifier'] == "q7.1" and ans["FreeText"] == "true":
            answer["q7"] = 1
        elif  ans['QuestionIdentifier'] == "q7.2" and ans["FreeText"] == "true":
            answer["q7"] = 2
        elif  ans['QuestionIdentifier'] == "q7.3" and ans["FreeText"] == "true":
            answer["q7"] = 3
        elif  ans['QuestionIdentifier'] == "q7.4" and ans["FreeText"] == "true":
            answer["q7"] = 4
        elif  ans['QuestionIdentifier'] == "q7.5" and ans["FreeText"] == "true":
            answer["q7"] = 5
            
        elif  ans['QuestionIdentifier'] == "q8.1" and ans["FreeText"] == "true":
            answer["q8"] = 1
        elif  ans['QuestionIdentifier'] == "q8.2" and ans["FreeText"] == "true":
            answer["q8"] = 2
        elif  ans['QuestionIdentifier'] == "q8.3" and ans["FreeText"] == "true":
            answer["q8"] = 3
        elif  ans['QuestionIdentifier'] == "q8.4" and ans["FreeText"] == "true":
            answer["q8"] = 4
        elif  ans['QuestionIdentifier'] == "q8.5" and ans["FreeText"] == "true":
            answer["q8"] = 5
            
        elif  ans['QuestionIdentifier'] == "q9.1" and ans["FreeText"] == "true":
            answer["q9"] = 1
        elif  ans['QuestionIdentifier'] == "q9.2" and ans["FreeText"] == "true":
            answer["q9"] = 2
        elif  ans['QuestionIdentifier'] == "q9.3" and ans["FreeText"] == "true":
            answer["q9"] = 3
        elif  ans['QuestionIdentifier'] == "q9.4" and ans["FreeText"] == "true":
            answer["q9"] = 4
        elif  ans['QuestionIdentifier'] == "q9.5" and ans["FreeText"] == "true":
            answer["q9"] = 5
            
        elif  ans['QuestionIdentifier'] == "q10.1" and ans["FreeText"] == "true":
            answer["q10"] = 1
        elif  ans['QuestionIdentifier'] == "q10.2" and ans["FreeText"] == "true":
            answer["q10"] = 2
        elif  ans['QuestionIdentifier'] == "q10.3" and ans["FreeText"] == "true":
            answer["q10"] = 3
        elif  ans['QuestionIdentifier'] == "q10.4" and ans["FreeText"] == "true":
            answer["q10"] = 4
        elif  ans['QuestionIdentifier'] == "q10.5" and ans["FreeText"] == "true":
            answer["q10"] = 5
            
        elif  ans['QuestionIdentifier'] == "q11.1" and ans["FreeText"] == "true":
            answer["q11"] = 1
        elif  ans['QuestionIdentifier'] == "q11.2" and ans["FreeText"] == "true":
            answer["q11"] = 2
        elif  ans['QuestionIdentifier'] == "q11.3" and ans["FreeText"] == "true":
            answer["q11"] = 3
        elif  ans['QuestionIdentifier'] == "q11.4" and ans["FreeText"] == "true":
            answer["q11"] = 4
        elif  ans['QuestionIdentifier'] == "q11.5" and ans["FreeText"] == "true":
            answer["q11"] = 5
            
        elif  ans['QuestionIdentifier'] == "q12.1" and ans["FreeText"] == "true":
            answer["q12"] = 1
        elif  ans['QuestionIdentifier'] == "q12.2" and ans["FreeText"] == "true":
            answer["q12"] = 2
        elif  ans['QuestionIdentifier'] == "q12.3" and ans["FreeText"] == "true":
            answer["q12"] = 3
        elif  ans['QuestionIdentifier'] == "q12.4" and ans["FreeText"] == "true":
            answer["q12"] = 4
        elif  ans['QuestionIdentifier'] == "q12.5" and ans["FreeText"] == "true":
            answer["q12"] = 5

        elif  ans['QuestionIdentifier'] == "q13.1" and ans["FreeText"] == "true":
            answer["q13"] = 1
        elif  ans['QuestionIdentifier'] == "q13.2" and ans["FreeText"] == "true":
            answer["q13"] = 2
        elif  ans['QuestionIdentifier'] == "q13.3" and ans["FreeText"] == "true":
            answer["q13"] = 3
        elif  ans['QuestionIdentifier'] == "q13.4" and ans["FreeText"] == "true":
            answer["q13"] = 4
        elif  ans['QuestionIdentifier'] == "q13.5" and ans["FreeText"] == "true":
            answer["q13"] = 5

        elif  ans['QuestionIdentifier'] == "q14.1" and ans["FreeText"] == "true":
            answer["q14"] = 1
        elif  ans['QuestionIdentifier'] == "q14.2" and ans["FreeText"] == "true":
            answer["q14"] = 2
        elif  ans['QuestionIdentifier'] == "q14.3" and ans["FreeText"] == "true":
            answer["q14"] = 3
        elif  ans['QuestionIdentifier'] == "q14.4" and ans["FreeText"] == "true":
            answer["q14"] = 4
        elif  ans['QuestionIdentifier'] == "q14.5" and ans["FreeText"] == "true":
            answer["q14"] = 5

        elif  ans['QuestionIdentifier'] == "q15.1" and ans["FreeText"] == "true":
            answer["q15"] = 1
        elif  ans['QuestionIdentifier'] == "q15.2" and ans["FreeText"] == "true":
            answer["q15"] = 2
        elif  ans['QuestionIdentifier'] == "q15.3" and ans["FreeText"] == "true":
            answer["q15"] = 3
        elif  ans['QuestionIdentifier'] == "q15.4" and ans["FreeText"] == "true":
            answer["q15"] = 4
        elif  ans['QuestionIdentifier'] == "q15.5" and ans["FreeText"] == "true":
            answer["q15"] = 5

        elif  ans['QuestionIdentifier'] == "q16.1" and ans["FreeText"] == "true":
            answer["q16"] = 1
        elif  ans['QuestionIdentifier'] == "q16.2" and ans["FreeText"] == "true":
            answer["q16"] = 2
        elif  ans['QuestionIdentifier'] == "q16.3" and ans["FreeText"] == "true":
            answer["q16"] = 3
        elif  ans['QuestionIdentifier'] == "q16.4" and ans["FreeText"] == "true":
            answer["q16"] = 4
        elif  ans['QuestionIdentifier'] == "q16.5" and ans["FreeText"] == "true":
            answer["q16"] = 5
    return answer

In [58]:
# resultPath = "result/result-25-11-2019-11-50-33.json" # 90 ass complete
resultPath = "result/result-25-11-2019-17-09-43.json"

with open(resultPath, 'r') as f:
    results = json.load(f)

workerIDs = []
if os.path.exists('workerIDs.json'):
    with open('workerIDs.json') as json_file:
        workerIDs = json.load(json_file)
numOfAnswers = {}
totalAnswers = 0
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']

    # Get a list of the Assignments that have been submitted by Workers
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )

    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)

    answers = []
    for assignment in assignments:
    
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        accept_time = assignment['AcceptTime']
        submit_time = assignment['SubmitTime']
        deltaTime = submit_time-accept_time  
        
        if worker_id not in workerIDs:
            workerIDs.append(worker_id)
        
        if deltaTime.total_seconds() > 60:
            # Retrieve the value submitted by the Worker from the XML
            answer_dict = xmltodict.parse(assignment['Answer'])
#             print(answer_dict)
            answer = getAnsewer(answer_dict)
            answer['duration'] = deltaTime.total_seconds()
            answer['workerId'] = worker_id
            answer['assignmentId'] = assignment_id
    #         print (answer)
            answers.append(answer)

            # Approve the Assignment (if it hasn't already been approved)
            if assignment['AssignmentStatus'] == 'Submitted':
                client.approve_assignment(
                    AssignmentId=assignment_id,
                    OverrideRejection=False
                )
        else:
            print('Reject assignment= {} with workerid={} and hitid={}'.format(assignment_id,worker_id,item['hit_id']))
            client.reject_assignment(
                AssignmentId=assignment_id,
                RequesterFeedback='You did not finish the assignment properly'
            )
    numOfAnswers[item['hit_id']] = len(answers)
    totalAnswers += len(answers)
    # Add the answers that have been retrieved for this item
    item['answers'] = answers

with open('workerIDs.json', 'w') as outfile:
    json.dump(workerIDs, outfile)

head, tail = os.path.split(resultPath)
filename = tail.split(".")[0]    
with open('result/{}-ans.json'.format(filename), 'w') as outfile:
    json.dump(results, outfile)
    
print ("Total Answers = {}".format(totalAnswers))
print(json.dumps(numOfAnswers,indent=2))

Total Answers = 96
{
  "356TQKY9XGACR0WGW1TO0WVY6YT78P": 5,
  "3Y7LTZE0YUZT979ZIZMCN86MDIXUZJ": 8,
  "3QE4DGPGBSOU1SKFD175PXSM3FG4GI": 6,
  "34OWYT6U3XU9UPWMKK3ZRHGI4KM9IB": 7,
  "39I4RL8QGKU81OFQX7PNAX4NIFT4HH": 8,
  "359AP8GAGHXE33MTDD9T2IRHWPEC70": 6,
  "3T5ZXGO9DF11HE2I1Q27D79XAXRZQJ": 6,
  "3421H3BM9BU5P0GS22OZ3IVIHZB9JN": 6,
  "3EAWOID6MUAWK1S9JVJPDV5J9INV0H": 7,
  "30U1YOGZGB9ARTCZ1C2FHF5V432SD2": 6,
  "3RDTX9JRTZEC55FQ1TZ20SQ9Q4B79E": 5,
  "33K3E8REWX866F27EEXMUV8M8R7X89": 9,
  "3GITHABACZYQ86MEWR1CM24LKD62NS": 4,
  "375VSR8FVXM1TFHIE5R7IXJ26JTZRK": 6,
  "3DWGDA5POGHWRQDRFENPR0OTEATV1K": 7
}


In [52]:
with open('result/{}-ans.json'.format(filename), 'r') as f:
    results = json.load(f)
    
if not os.path.exists("csv_output/"):
    os.makedirs("csv_output/")
    
with open("csv_output/{}.csv".format(filename), "w", newline='') as output:
    f = csv.writer(output)

    # Write CSV Header, If you dont need that, remove this line
    f.writerow(["surveyId","hitId","AssignmentId",
                "workerId","age","gender","education","occupation","Hispanic","race","Political","zipcode","duration",
                "feelAboutAd",
                "q1","q2","q3","q4","q5","q6","q7","q8","q9","q10","q11","q12","q13","q14","q15","q16",
                "image1","image2","image3","image4","image5","image6","image7","image8","image9",
                "image10","image11","image12","image13","image14","image15","image16"])

    for item in results:
        for answer in item["answers"]:
            f.writerow([item["id"],item["hit_id"],answer["assignmentId"],
                        answer["workerId"],answer["age"],answer["gender"],
                        answer["education"],answer["occupation"],answer["Hispanic"],
                        answer["race"],answer["Political"],answer["zipCode"],answer["duration"],
                        answer["feelAboutAd"],
                        answer["q1"],answer["q2"],answer["q3"],answer["q4"],answer["q5"],answer["q6"],
                        answer["q7"],answer["q8"],answer["q9"],answer["q10"],answer["q11"],answer["q12"],
                        answer["q13"],answer["q14"],answer["q15"],answer["q16"],
                        item["image1"],item["image2"],item["image3"],item["image4"],item["image5"],
                        item["image6"],item["image7"],item["image8"],item["image9"],item["image10"],
                        item["image11"],item["image12"],item["image13"],item["image14"],item["image15"],
                        item["image16"]])

    